In [66]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from datetime import datetime
from config import weather_api_key
from citipy import citipy
import requests

#create the url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

In [67]:
#create a set of random latitudes and longitudes as a combination.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [68]:
#add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [69]:
# create a list for holding the cities
cities = []

#identify the nearest city for each latitude and longitude combo. 
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #if the city is unique then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count 
len(cities)

628

In [70]:
#create an empty list to hold the weather data. 
city_data = []

#print the beginning of the logging. 
print("Beginning Data Retrieval   ")
print("---------------------------")

#create counters
record_count = 1
set_count = 1

#loop through the cities in out list
for i, city in enumerate(cities):

    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count =1
    #create endpoint URL with each city 
    city_url = url + "&q=" + city.replace(" ","+")

    #log the URS, record, and set the numbers and the city. 
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    #add 1 to the record_count 
    record_count +=1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        continue

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval   
---------------------------
Processing Record 1 of Set 1 | ouadda
Processing Record 2 of Set 1 | qui nhon
Processing Record 3 of Set 1 | vaini
Processing Record 4 of Set 1 | sitka
Processing Record 5 of Set 1 | parkes
Processing Record 6 of Set 1 | babanusah
City not found. Skipping...
Processing Record 7 of Set 1 | porto velho
Processing Record 8 of Set 1 | merauke
Processing Record 9 of Set 1 | arraial do cabo
Processing Record 10 of Set 1 | nikolskoye
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | kapaa
Processing Record 13 of Set 1 | kazalinsk
City not found. Skipping...
Processing Record 14 of Set 1 | bredasdorp
Processing Record 15 of Set 1 | shakawe
Processing Record 16 of Set 1 | hilo
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | punta arenas
Processing Record 19 of Set 1 | slupsk
Processing Record 20 of Set 1 | avarua
Processing Record 21 of Set 1 | chimoio
Processing Record 22 of Set 1 | guerre

In [71]:
len(city_data)

579

In [73]:
#convert the list to a df
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ouadda,8.0777,22.4007,79.39,18,98,2.75,CF,2021-04-07 22:57:44
1,Qui Nhon,13.7667,109.2333,74.52,89,92,5.64,VN,2021-04-07 22:57:46
2,Vaini,-21.2000,-175.2000,80.60,69,20,6.91,TO,2021-04-07 22:57:46
3,Sitka,57.0531,-135.3300,39.20,65,75,14.97,US,2021-04-07 22:57:46
4,Parkes,-33.1333,148.1833,69.57,56,4,3.91,AU,2021-04-07 22:57:47


In [82]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Ouadda,CF,2021-04-07 22:57:44,8.0777,22.4007,79.39,18,98,2.75
1,Qui Nhon,VN,2021-04-07 22:57:46,13.7667,109.2333,74.52,89,92,5.64
2,Vaini,TO,2021-04-07 22:57:46,-21.2000,-175.2000,80.60,69,20,6.91
3,Sitka,US,2021-04-07 22:57:46,57.0531,-135.3300,39.20,65,75,14.97
4,Parkes,AU,2021-04-07 22:57:47,-33.1333,148.1833,69.57,56,4,3.91


In [83]:
#create the output file (CSV).
output_data_file = "weather_data/cities.csv"

#export the city_data into csv
city_data_df.to_csv(output_data_file, index_label="City_ID")